# The code is to run the RGD tomography method 
1. prepare the quantum state
2. do the measurement
3. do the RGD optimization method to reconstruct the state

In [1]:
import numpy as np

#import qutip as qu
#from states import GHZState, HadamardState, RandomState
from qibo_states import GHZState, HadamardState, RandomState
from qibo import quantum_info

import methodsRGD_core
import methodsMiFGD_core

import measurements
import projectors

In [2]:
n = 3
labels = projectors.generate_random_label_list(20, n)
num_labels  = len(labels)

In [3]:
Nr = 1 

#state   = GHZState(n)
#state   = HadamardState(n)
state   = RandomState(n)

target_density_matrix = state.get_state_matrix()
target_state          = state.get_state_vector()  

[Qibo 0.2.12|INFO|2024-09-28 21:57:42]: Using numpy backend on /CPU:0
[Qibo 0.2.12|INFO|2024-09-28 21:57:42]: Using numpy backend on /CPU:0


### Do the shot measurement

In [4]:
state.create_circuit()
data_dict_list = state.execute_measurement_circuits(labels)
#print(data_dict_list)

data_dict = {}
for ii in range(num_labels):
    label = data_dict_list[ii]['label']
    data_dict[label] = data_dict_list[ii]['count_dict'] 
        
measurement_list = measurements.MeasurementStore.calc_measurement_list(labels, data_dict)

[Qibo 0.2.12|INFO|2024-09-28 21:57:42]: Using numpy backend on /CPU:0


    -------------------         start calc measurment_list        ------------------  

       ****   directly calc & save measurement_list  
       ****   len(label_list) = 20,  len(data_dict) = 20

    -------------         DONE of calculating measurment_list       ------------------  



### construct Pauli matrix Projectors
1. construct the Pauli matrix
2. calculate the exact coefficient

In [5]:
projector_store_path = './testData/qibo'
projector_store = projectors.ProjectorStore(labels)

# 1. construct the sampled Pauli operators
num_cpus, saveP_bulk, Partition_Pj = projector_store.mpPool_map(projector_store_path)
projector_dict = projectors.ProjectorStore.load_PoolMap(projector_store_path, labels)

# 2. calculate the exact coefficients for sampled Pauli operators
projector_list = [projector_dict[label] for label in labels]
yProj_Exact = methodsRGD_core.Amea(projector_list, target_density_matrix, num_labels, 1) # argv[-1] = coef


      Partion_Pj = 0  -->  No partion in Pj

  ***  Pj_file    = ./testData/qibo/Pj_list.pickle  is saved (i.e. dump)
  ***  label_file = ./testData/qibo/labels.pickle   is saved

     pool.map  COMPLETED by #CPU = 12 

     [projectors] num_cpus = 12,  saved bulk Pj = 1 



In [6]:
params_dict = { 'Nr': Nr,
                'target_DM': target_density_matrix,
                'labels': labels,
                'measurement_list': measurement_list,
                'projector_list': projector_list,
                'num_iterations': 150,
                'convergence_check_period': 1 
}

### DO the RGD tomography optimization method

In [7]:
Ch_svd = -1                 #   choice for initial SVD  (0: LA.svd, 1: svds;  2: power_Largest_EigV, -1: rSVD)
InitX_RGD = 1;              #   method of choosing initial X0

worker = methodsRGD_core.BasicWorkerRGD(params_dict)
worker.computeRGD(InitX_RGD, Ch_svd)


  self.coef = 0.6324555320336759
  InitX     = 1
 *********   using randomized-SVD  to construct  X0 = uk @ sDiag @ vkh  **************************
  ***   0-th (A A*) done:   Time --> ATQ: 0.0010030269622802734,  AG: 0.0   ***
  ***   1-th (A A*) done:   Time --> ATQ: 0.0010046958923339844,  AG: 0.0009984970092773438   ***
  ***   2-th (A A*) done:   Time --> ATQ: 0.0005080699920654297,  AG: 0.0   ***
  ***   3-th (A A*) done:   Time --> ATQ: 0.0,  AG: 0.001009225845336914   ***
  ***   4-th (A A*) done:   Time --> ATQ: 0.0,  AG: 0.0   ***
  ***   5-th (A A*) done:   Time --> ATQ: 0.0010001659393310547,  AG: 0.0   ***
  ***   6-th (A A*) done:   Time --> ATQ: 0.0,  AG: 0.0010068416595458984   ***
  ***   7-th (A A*) done:   Time --> ATQ: 0.0,  AG: 0.0   ***
  ***   8-th (A A*) done:   Time --> ATQ: 0.0010056495666503906,  AG: 0.0   ***
  ***   9-th (A A*) done:   Time --> ATQ: 0.0009982585906982422,  AG: 0.0   ***
  ***   10-th (A A*) done:   Time --> ATQ: 0.0009999275207519531,  AG: 

### Plot the result

In [8]:
worker.Target_Err_Xk

len(worker.step_Time)


43

In [9]:
len(worker.Target_Err_Xk)

43